<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/%E4%BD%BF%E7%94%A8ChatHaruhi54K%E7%94%9F%E6%88%90messages%E5%BD%A2%E5%BC%8F%E6%95%B0%E6%8D%AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [ ] RoleLLM的数据
- [ ] 其他中文的hf的一些数据
- [ ] ChatHaruhi54K本身的数据

这里我们假设输入数据已经组织成了hf格式

可以有效载入到chatbot中

In [1]:
!pip -q install openai tiktoken langchain datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.3/222.3 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.9/218.9 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 577.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

组织数据

In [34]:
from datasets import load_dataset

dataset = load_dataset("silk-road/ChatHaruhi-54K-Role-Playing-Dialogue")

role2tuple = {}

from tqdm import tqdm
for data in tqdm( dataset['train'] ):
    agent_role = data['agent_role']
    if agent_role not in role2tuple:
        role2tuple[agent_role] = []

    user_role = data['user_role']
    user_question = data['user_question']

    query = user_role + ":" + user_question + ""

    agent_response = data['agent_response']

    target = agent_role + ":" + agent_response + ""

    # role2tuple[agent_role].append((query, target))

    messages = [
        {"role":"user","content":query,'user_role':user_role,'user_question':user_question},
        {"role":"assistent","content":target}
    ]


    more_dialogues = data['more_dialogues']

    if len(more_dialogues) > 0:
        n = len( more_dialogues )
        for i in range(n):

            sent = more_dialogues[i]

            if sent.startswith(agent_role):
                messages.append( {"role":"assistent", "content":sent} )
            else:
                messages.append( {"role":"user", "content":sent} )

    role2tuple[agent_role].append(messages)



100%|██████████| 54726/54726 [00:04<00:00, 11434.39it/s]


In [ ]:
for key in role2tuple:
    print(key, len(role2tuple[key]))

tuple数据会保留在role2tuple

In [4]:
print(role2tuple['Harry'][100])

[{'role': 'user', 'content': 'Ginny:「Did I hear right? You’ve been taking orders from something someone wrote in a book, Harry?」'}, {'role': 'assistent', 'content': '「It’s nothing. It’s not like, you know, Riddle’s diary. It’s just an old textbook someone’s scribbled on.」'}]


训练的参数设定

In [5]:
%cd /content
!rm -rf /content/Haruhi-2-Dev
!git clone https://github.com/LC1332/Haruhi-2-Dev
%cd /content/Haruhi-2-Dev

/content
Cloning into 'Haruhi-2-Dev'...
remote: Enumerating objects: 1046, done.
remote: Counting objects: 100% (353/353), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 1046 (delta 244), reused 306 (delta 210), pack-reused 693
Receiving objects: 100% (1046/1046), 106.35 MiB | 48.19 MiB/s, done.
Resolving deltas: 100% (574/574), done.
/content/Haruhi-2-Dev


In [6]:
role_name_Haruhiu = {'汤师爷': 'tangshiye', 'tangshiye': 'tangshiye', 'Tangshiye': 'tangshiye',
                     '慕容复': 'murongfu', 'murongfu': 'murongfu', 'Murongfu': 'murongfu',
                     '李云龙': 'liyunlong', 'liyunlong': 'liyunlong', 'Liyunlong': 'liyunlong',
                     'Luna': 'Luna', '王多鱼': 'wangduoyu', 'wangduoyu': 'wangduoyu',
                     'Wangduoyu': 'wangduoyu', 'Ron': 'Ron', '鸠摩智': 'jiumozhi',
                     'jiumozhi': 'jiumozhi', 'Jiumozhi': 'jiumozhi', 'Snape': 'Snape',
                     '凉宫春日': 'haruhi', 'haruhi': 'haruhi', 'Haruhi': 'haruhi',
                     'Malfoy': 'Malfoy', '虚竹': 'xuzhu', 'xuzhu': 'xuzhu',
                     'Xuzhu': 'xuzhu', '萧峰': 'xiaofeng',
                     'xiaofeng': 'xiaofeng', 'Xiaofeng': 'xiaofeng', '段誉': 'duanyu',
                     'duanyu': 'duanyu', 'Duanyu': 'duanyu', 'Hermione': 'Hermione',
                     'Dumbledore': 'Dumbledore', '王语嫣': 'wangyuyan', 'wangyuyan':
                     'wangyuyan', 'Wangyuyan': 'wangyuyan', 'Harry': 'Harry',
                     'McGonagall': 'McGonagall', '白展堂': 'baizhantang',
                     'baizhantang': 'baizhantang', 'Baizhantang': 'baizhantang',
                     '佟湘玉': 'tongxiangyu', 'tongxiangyu': 'tongxiangyu',
                     'Tongxiangyu': 'tongxiangyu', '郭芙蓉': 'guofurong',
                     'guofurong': 'guofurong', 'Guofurong': 'guofurong', '流浪者': 'wanderer',
                     'wanderer': 'wanderer', 'Wanderer': 'wanderer', '钟离': 'zhongli',
                     'zhongli': 'zhongli', 'Zhongli': 'zhongli', '胡桃': 'hutao', 'hutao': 'hutao',
                     'Hutao': 'hutao', 'Sheldon': 'Sheldon', 'Raj': 'Raj',
                     'Penny': 'Penny', '韦小宝': 'weixiaobao', 'weixiaobao': 'weixiaobao',
                     'Weixiaobao': 'weixiaobao', '乔峰': 'qiaofeng', 'qiaofeng': 'qiaofeng',
                     'Qiaofeng': 'qiaofeng', '神里绫华': 'ayaka', 'ayaka': 'ayaka',
                     'Ayaka': 'ayaka', '雷电将军': 'raidenShogun', 'raidenShogun': 'raidenShogun',
                     'RaidenShogun': 'raidenShogun', '于谦': 'yuqian', 'yuqian': 'yuqian',
                     'Yuqian': 'yuqian', 'Professor McGonagall': 'McGonagall',
                     'Professor Dumbledore': 'Dumbledore'}

In [ ]:
!ls

ChatHaruhi  data  LICENSE  notebook  Readme.md


In [12]:
from ChatHaruhi import ChatHaruhi

from tqdm import tqdm

role_en2bots = {}

for ai_role_en in tqdm( role_name_Haruhiu.values() ):
    if ai_role_en in role_en2bots:
        continue

    role_en2bots[ai_role_en] = ChatHaruhi(role_name = ai_role_en, llm = 'foo')

100%|██████████| 76/76 [00:42<00:00,  1.78it/s]


In [13]:
count = 0

for role in role2tuple:
    if role in role_name_Haruhiu:
        en_name = role_name_Haruhiu[role]
        if en_name in role_en2bots:
            count+= len(role2tuple[role])

print(count)

52017


In [24]:
K_SEARCH = 3
MAX_LEN_STORY = 1000 #这个是按照token算的
MAX_LEN_HISTORY = 1200 # count with token
source = "Haruhi52K"

In [ ]:
from datasets import load_dataset
from tqdm import tqdm

save_datas = []

# for role_name in tqdm(role_from_roleLLM):

for role_name_zh in role2tuple.keys():
    if role_name_zh not in role_name_Haruhiu:
        continue
    role_name_en = role_name_Haruhiu[role_name_zh]

    chatbot = role_en2bots[role_name_en]


    chatbot.k_search = K_SEARCH
    chatbot.max_len_story = MAX_LEN_STORY
    chatbot.max_len_history = MAX_LEN_HISTORY

    all_messages = role2tuple[role_name_zh]

    for messages in tqdm(all_messages):
        query = messages[0]['content']
        user_role = messages[0]['user_role']
        user_question = messages[0]['user_question'].strip(" 「」")
        prompt_messages = chatbot.generate_messages(user_question, user_role )

        conversations = []
        system_prompt = ""
        for msg in prompt_messages[:-1]:
            role = msg["role"]
            content = msg["content"]
            if role == "AI":
                conversations.append({"from":"gpt","value":content})
            elif role == "System":
                if system_prompt == "":
                    system_prompt = content
            elif role == "User":
                conversations.append({"from":"human","value":content})

        for msg in messages:
            content = msg["content"]
            if msg["role"] == "user":
                conversations.append({"from":"human","value":content})
            else:
                conversations.append({"from":"gpt","value":content})


        save_data = {
            "source":source,
            "conversations":conversations,
            "system":system_prompt
        }
        save_datas.append(save_data)


 29%|██▉       | 351/1216 [00:44<01:37,  8.89it/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import json
import random
def sample_and_save( save_datas, save_name ,sample_n=50, save_folder = "/content/drive/MyDrive/CardBuild/HaruhiZero"):
    file_name = os.path.join( save_folder, save_name + ".jsonl" )
    with open( file_name, "w", encoding="utf-8" ) as f:
        for data in save_datas:
            f.write( json.dumps( data, ensure_ascii=False ) + "\n" )

    sample_name = os.path.join( "/content/", save_name + "_sample.jsonl" )
    with open( sample_name, "w", encoding="utf-8" ) as f:
        random_50 = random. sample( save_datas, sample_n )
        for data in random_50:
            f.write( json.dumps( data, ensure_ascii=False ) + "\n" )

In [ ]:
sample_and_save( save_datas, "Haruhi52K" , sample_n = 50)

In [41]:
save_data = save_datas[3]
for msg in save_data["conversations"]:
    print(msg)

{'from': 'human', 'value': "Classic scenes for the role are as follows:\nTogether:「Not at all.」\nRaj:「Papa, please don't start.」\nDr. Koothrappali:「God, it's just a question, he's so sensitive.」\nRaj:「Okay, that's my life, that's my friends, good to see you, say goodbye.」\n###\nDr. Koothrappali:「Bye bye.」\nRaj:「I don't believe it.」\n###\nLeonard:「This isn't a substitution, it's a reduction.」\nChen:「Okay, no reductions.」\nLeonard:「Fine, bring us three orders of dumplings, that's twelve, we'll each have four.」\nRaj:「That works.」\n###\n"}
{'from': 'human', 'value': "Leonard:「Take it, Raj. It's more than I’ve ever gotten.」"}
{'from': 'gpt', 'value': "Raj:「Oh, I couldn't.」"}
{'from': 'human', 'value': 'Leonard:「No, no, no, you should.」'}
{'from': 'gpt', 'value': 'Raj:「Okay, I will.」'}
{'from': 'human', 'value': "Sheldon:「I'm sorry, are we taking turns talking now?」"}
